<a href="https://colab.research.google.com/github/mdzikrim/DeepLearning/blob/main/CNN_Model_SVHN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CNN PyTorch

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = datasets.SVHN(root='./data', split='train', transform=transform, download=True)
testset = datasets.SVHN(root='./data', split='test', transform=transform, download=True)

trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=False)


100%|██████████| 182M/182M [00:03<00:00, 58.1MB/s]
100%|██████████| 64.3M/64.3M [00:01<00:00, 52.3MB/s]


In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 8 * 8, 256), nn.ReLU(),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        return self.fc(self.conv(x))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN().to(device)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(20):
    model.train()
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        loss = criterion(model(images), labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1} done")


Epoch 1 done
Epoch 2 done
Epoch 3 done
Epoch 4 done
Epoch 5 done
Epoch 6 done
Epoch 7 done
Epoch 8 done
Epoch 9 done
Epoch 10 done
Epoch 11 done
Epoch 12 done
Epoch 13 done
Epoch 14 done
Epoch 15 done
Epoch 16 done
Epoch 17 done
Epoch 18 done
Epoch 19 done
Epoch 20 done


In [ ]:
model.eval()
y_true, y_pred, y_probs = [], [], []

with torch.no_grad():
    for images, labels in testloader:
        images = images.to(device)
        outputs = model(images)
        probs = torch.softmax(outputs, dim=1).cpu().numpy()
        preds = np.argmax(probs, axis=1)

        y_probs.extend(probs)
        y_pred.extend(preds)
        y_true.extend(labels.numpy())

acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred, average='macro')
rec = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')
auc = roc_auc_score(np.eye(10)[y_true], y_probs, multi_class='ovr')

print(f"Akurasi : {acc:.4f}")
print(f"Presisi : {prec:.4f}")
print(f"Recall  : {rec:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"AUC     : {auc:.4f}")


Akurasi : 0.8877
Presisi : 0.8763
Recall  : 0.8790
F1-Score: 0.8771
AUC     : 0.9874


#CNN Tensorflow

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np


In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'svhn_cropped',
    split=['train', 'test'],
    as_supervised=True,
    with_info=True
)

def normalize_img(image, label):
    return tf.cast(image, tf.float32) / 255.0, label

batch_size = 64

ds_train = ds_train.map(normalize_img).shuffle(10000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.map(normalize_img).batch(batch_size).prefetch(tf.data.AUTOTUNE)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/svhn_cropped/incomplete.QHAWAY_3.1.0/svhn_cropped-train.tfrecord*...:   0%…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/svhn_cropped/incomplete.QHAWAY_3.1.0/svhn_cropped-test.tfrecord*...:   0%|…

Generating extra examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/svhn_cropped/incomplete.QHAWAY_3.1.0/svhn_cropped-extra.tfrecord*...:   0%…

Dataset svhn_cropped downloaded and prepared to /root/tensorflow_datasets/svhn_cropped/3.1.0. Subsequent calls will reuse this data.


In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(10, activation='softmax')
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(ds_train, epochs=15, validation_data=ds_test)

Epoch 1/15
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 22s 14ms/step - accuracy: 0.4732 - loss: 1.5340 - val_accuracy: 0.8494 - val_loss: 0.5481
Epoch 2/15
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 17s 14ms/step - accuracy: 0.8442 - loss: 0.5363 - val_accuracy: 0.8666 - val_loss: 0.4702
Epoch 3/15
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 17s 13ms/step - accuracy: 0.8649 - loss: 0.4496 - val_accuracy: 0.8763 - val_loss: 0.4290
Epoch 4/15
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 17s 14ms/step - accuracy: 0.8815 - loss: 0.3965 - val_accuracy: 0.8762 - val_loss: 0.4267
Epoch 5/15
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 16s 13ms/step - accuracy: 0.8931 - loss: 0.3587 - val_accuracy: 0.8834 - val_loss: 0.4095
Epoch 6/15
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 16s 13ms/step - accuracy: 0.9005 - loss: 0.3333 - val_accuracy: 0.8876 - val_loss: 0.4072
Epoch 7/15
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 15s 12ms/step - accuracy: 0.9071 - loss: 0.3088 - val_accuracy: 0.8922 - val_loss: 0.3909
Epoch 8/15
1145/1145 ━━━━━━━━━━━━━━━━━━━━ 15s 12ms/step - accuracy: 0.9131 -

In [ ]:
# Prediksi semua sekaligus
y_probs = model.predict(ds_test)
y_pred = np.argmax(y_probs, axis=1)

# Ambil label asli
y_true = np.concatenate([labels.numpy() for _, labels in ds_test])

# Hitung metrik
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

y_true_onehot = tf.keras.utils.to_categorical(y_true, 10)
auc = roc_auc_score(y_true_onehot, y_probs, multi_class='ovr', average='macro')

print(f"Akurasi : {accuracy:.4f}")
print(f"Presisi : {precision:.4f}")
print(f"Recall  : {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"AUC     : {auc:.4f}")


407/407 ━━━━━━━━━━━━━━━━━━━━ 8s 19ms/step
Akurasi : 0.9013
Presisi : 0.8957
Recall  : 0.8899
F1-Score: 0.8924
AUC     : 0.9900
